### 题目描述
请设计一个简易数据库系统，支持三个命令：

- CREATE(int tableId, int colNum, string keys) ：创建一个表，其id为tableId；如果该表已经存在，则不做任何处理。
- colNum 为该表列的数量，列名由单字母表示，并从字母 a 开始、按 a-z 顺序编号，例如colNum=3，表示有三列，列名分别为字母 a b c；
- keys 表示该表的主键（指的是一列或多列的组合，其值能唯一地标识表中的每一行），keys中每个字符代表一列，如 keys=ba表示主键为b列和a列的组合。
- INSERT(int tableId, int[] values) ：添加一条记录：

- values 每个元素按顺序一一对应每列的值。
若主键冲突，则不做任何处理。
- SELECT(int tableId, string[] conditions) ：根据条件查询记录，并按主键升序输出结果集：

- conditions 每个元素代表一个条件，格式如a=25，条件仅为等于（不存在大于、小于等情况）；conditions 各元素间仅为 and 关系。
主键升序：按keys中列出现的顺序依次进行排序，每列按值大小升序输出。如keys=ba，先按b列升序排序，若b列值相同则进一步按a列升序排序。
现给出一批命令语句，请按输入顺序解析与执行，并输出每个SELECT 命令的结果集（CREATE 和 INSERT 命令无输出）

### 解答要求

时间限制：1000ms, 内存限制：256MB
输入
首行为整数 num, 表示其后输入的命令行数，取值范围[1,30000]。
之后每行为一条命令，3种命令的输入格式分别为：

- CREATE 命令，格式为：CREATE:tableId,colNum,keys
- tableId 取值范围[1,10000]。
- colNum 取值范围[1,26]。
- keys 为仅由小写字母组成的字符串，每个字符都不重复且在列名范围内，长度范围[1,26]。
- INSERT 命令，格式为：INSERT:tableId,value1 value2 ...
- value 的取值范围 [0,10000]
- SELECT 命令，格式为：SELECT:tableId,condition1 and condition2 ...
每个condition 的格式如a=25
查询条件有若干个，小于等于 colNum，且不重复。
用例保证 INSERT 和 SELECT命令中tableId所代表的表是已经创建的
每种命令最多10000条命令

输出
- 逐个输出每个 SELECT 命令的结果集，每行输出表中所有列的值、且按列的顺序输出。
如果某个SELECT命令的查询结果为空，此SELECT输出empty。

注：CREATE 和 INSERT 的执行结果不需要输出
```
样例
输入样例 1

5
CREATE:1,3,a
INSERT:1,2 3 7
INSERT:1,4 5 6
INSERT:1,3 4 6
SELECT:1,b=5 and c=6
输出样例 1

4 5 6
提示样例 1
CREATE:1,3,a 创建一个id为1的数据库表，有3列（列名分别为a,b,c），a列为主键
INSERT:1,2 3 7 向数据库表1插入一条记录，其值分别为 2 3 7 （即a=2,b=3,c=7）
INSERT:1,4 5 6 向数据库表1插入一条记录，其值分别为 4 5 6
INSERT:1,3 4 6 向数据库表1插入一条记录，其值分别为 3 4 6
SELECT:1,b=5 and c=6 查询数据库表1的内容，条件是b=5并且c=6，符合要求的记录只有4 5 6 这一条



输入样例 2

10
CREATE:2,4,ba
INSERT:2,9 23 47 41
CREATE:3,3,ca
INSERT:2,10 20 47 99
INSERT:2,10 23 47 58
INSERT:3,9 555 47
INSERT:3,9 23 47
SELECT:2,c=47
SELECT:2,a=9 and b=555
SELECT:3,a=9
输出样例 2

10 20 47 99
9 23 47 41
10 23 47 58
empty
9 555 47
提示样例 2
CREATE:2,4,ba 创建一个id为2的数据库表，有4列，主键为b列和a列的组合
CREATE:3,3,ca 创建一个id为3的数据库表，有3列，主键为c列和a列的组合
INSERT:3,9 555 47 在表3中插入一条记录，其主键值为c=47 and a=9
INSERT:3,9 23 47 因为与表3中已有数据冲突(主键值也为c=47 and a=9)，所以插入失败
SELECT:2,c=47查询输出3条记录：按照主键升序输出（先按b列升序排序，若b列值相同则进一步按a列升序排序）
SELECT:2,a=9 and b=555 查询结果为空，所以输出empty
```


In [637]:
"""
Copyright (c) Huawei Technologies Co., Ltd. 2019-2021. All rights reserved.
Description: 上机编程认证
Note: 缺省代码仅供参考，可自行决定使用、修改或删除
"""

from typing import List
from string import ascii_lowercase


class Solution:
    def __init__(self):
        """
        注意到本题没有删除选项,用个列表出错主键，使劲往里面怼，不用担心删除问题
        """
        # 大小写编号
        self.ascii_num = {y: x for x, y in enumerate(ascii_lowercase)}
        # 表格记录
        self.table = {}

    def create(self, table_id: int, col_num: int, keys: str) -> None:
        """在此添加你的代码"""
        self.table[table_id] = {
            'sort_table': [],  # 此表专门用于排序
            'key': [self.ascii_num[key] for key in keys],
            'ket_value': []
        }

    def insert(self, table_id: int, values: List[int]) -> None:
        """在此添加你的代码"""
        keys = self.table[table_id]['key']
        ket_value = self.table[table_id]['ket_value']
        main_key = []
        for key in keys:
            main_key.append(values[key])
        # 无主键冲突插入，并排序
        if main_key not in ket_value:
            # 先插入排序表
            main_key_list = [values[key] for key in keys]
            self.table[table_id]['sort_table'].append([main_key_list, values])
            ket_value.append(main_key)
            self.table[table_id]['sort_table'].sort()

    def select(self, table_id: int, conditions: List[str]) -> List[int]:
        """在此添加你的代码"""
        table = self.table[table_id]['sort_table']
        table = [x[1] for x in table]
        conditions = [condition.split('=') for condition in conditions]
        conditions = [[self.ascii_num[key], int(value)] for key, value in conditions]
        res = []
        for lis in table:
            adds = True
            for condition in conditions:
                if lis[condition[0]] != condition[1]:
                    adds = False
                    break
            if adds:
                res.append(lis)
        return res if res else [['empty']]


In [638]:
sol = Solution()

```sql
CREATE:1,3,a
INSERT:1,2 3 7
INSERT:1,4 5 6
INSERT:1,3 4 6
SELECT:1,b=5 and c=6
```

In [639]:
sol.create(1,3,'a')

In [640]:
sol.insert(1,[2,3,7])

In [641]:
sol.insert(1,[4,5,6])

In [642]:
sol.insert(1,[3,4,6])

In [643]:
sol.select(1,['b=5','c=6'])

[[4, 5, 6]]

In [644]:
sol.table[1]

{'sort_table': [[[2], [2, 3, 7]], [[3], [3, 4, 6]], [[4], [4, 5, 6]]],
 'key': [0],
 'ket_value': [[2], [4], [3]]}